In [1]:
import iql

In [115]:
import pandas as pd

In [88]:
class get_account_queries(object):
    def __init__(self, start='90d', end='today'):
        self._start = start
        self._end = end
        self._js_accounts = ['us_jobseeker_usd_broadmatch_2_en',
                             'us_jobseeker_usd_exactmatch_5_en',
                             'us_jobseeker_usd_exactmatch_2_en',
                             'us_jobseeker_usd_exactmatch_0_en',
                             'us_jobseeker_usd_exactmatch_1_en',
                             'us_jobseeker_usd_broadmatch_1_en',
                             'us_jobseeker_usd_exactmatch_6_en',
                             'us_jobseeker_usd_geo_2_active',
                             'us_jobseeker_usd_broadmatch_0_en',
                             'us_jobseeker_usd_exactmatch_3_en',
                             'us_jobseeker_usd_geo_active',
                             'us_jobseeker_usd_exactmatch_4_en']
        self._dradis_accounts = ['us_dradis_usd_exactmatch_0_active',
                                 'us_dradis_usd_exactmatch_1_active',
                                 'us_dradis_sh_usd_0_active',
                                 'us_dradis_usd_main_active']
        self._dradis_raw = {}
        self._js_raw = {}  
        self._dradis_queries = {}
        self._js_queries = {}        
        
    def query_builder(self, account):
        qry = ("FROM semgoogle {start} {end} WHERE account_country='US' "
               "report_origin='SEARCH_QUERY_PERFORMANCE_REPORT' "
               "account_name='{account}' clicks>0 GROUP BY query SELECT "
               "clicks").format(start=self._start, end=self._end, account=account)
        return qry
 
    def get_account_raw_data(self, account):
        data = iql.dataframe(self.query_builder(account))
        return data
    
    def get_queries_raw(self, account_list):
        data = {}
        for account in account_list:
            data[account] = self.get_account_raw_data(account)
        return data
    
    def populate_raw_data(self):
        self._js_raw = self.get_queries_raw(self._js_accounts)
        self._dradis_raw = self.get_queries_raw(self._dradis_accounts)
    
    def get_account_data(self, account):
        data = iql.dataframe(self.query_builder(account))
        return set(data['query'].values)
    
    def get_queries(self, account_list):
        data = {}
        for account in account_list:
            data[account] = self.get_account_data(account)
        return data
    
    def populate_data(self):
        self._js_queries = self.get_queries(self._js_accounts)
        self._dradis_queries = self.get_queries(self._dradis_accounts)

In [89]:
a = get_account_queries()

In [90]:
a.populate_data()

In [91]:
len(a._js_queries)

12

In [92]:
len(a._dradis_queries)

4

In [93]:
set([1,2])|set([3])

{1, 2, 3}

In [94]:
a._dradis_queries.keys()

['us_dradis_usd_exactmatch_0_active',
 'us_dradis_usd_exactmatch_1_active',
 'us_dradis_sh_usd_0_active',
 'us_dradis_usd_main_active']

In [53]:
a._js_queries.keys()

['us_jobseeker_usd_exactmatch_3_en',
 'us_jobseeker_usd_broadmatch_2_en',
 'us_jobseeker_usd_broadmatch_0_en',
 'us_jobseeker_usd_exactmatch_2_en',
 'us_jobseeker_usd_exactmatch_5_en',
 'us_jobseeker_usd_broadmatch_1_en',
 'us_jobseeker_usd_exactmatch_1_en',
 'us_jobseeker_usd_geo_active',
 'us_jobseeker_usd_geo_2_active',
 'us_jobseeker_usd_exactmatch_4_en',
 'us_jobseeker_usd_exactmatch_6_en',
 'us_jobseeker_usd_exactmatch_0_en']

In [95]:
dd = set()
for i in a._dradis_queries:
    dd = dd | a._dradis_queries[i]
len(dd)

306600

In [96]:
jj = set()
for i in a._js_queries:
    jj = jj |a._js_queries[i]
len(jj)

3983966

In [97]:
common_queries = dd.intersection(jj)

In [99]:
len(common_queries)

5754

In [86]:
b = get_account_queries()

In [87]:
b.populate_raw_data()

In [102]:
#TODO: add click data
tmp = dict()
for i in b._dradis_raw:
    tmp.update(b._dradis_raw[i].set_index('query').to_dict()['clicks'])

In [127]:
#TODO: add click data
tmp_js = dict()
for i in b._js_raw:
    tmp_js.update(b._js_raw[i].set_index('query').to_dict()['clicks'])

In [110]:
final = dict()
for i in tmp:
    if i in common_queries:
        final[i] = tmp[i]

In [128]:
final_js = dict()
for i in tmp_js:
    if i in common_queries:
        final_js[i] = tmp_js[i]

In [118]:
xyz = pd.DataFrame(final.items(), columns=['query', 'clicks'])

In [121]:
xyz = xyz.sort_values('clicks', ascending=0)

In [129]:
xyz_js = pd.DataFrame(final_js.items(), columns=['query', 'clicks'])

In [139]:
xyz_js = xyz_js.sort_values('clicks', ascending=0)

In [130]:
#xyz[xyz.clicks>=10]

In [131]:
len(xyz)

5754

In [143]:
len(js_final)

5754

In [140]:
js_final = xyz_js.set_index('query')
dradis_final = xyz.set_index('query')

In [149]:
allx = js_final.merge(dradis_final, left_index=True, right_index=True, how='inner')

In [151]:
allx.columns = ['js_clicks', 'dradis_clicks']

In [152]:
allx['js_clicks/dradis_clicks'] = allx.js_clicks/allx.dradis_clicks

In [156]:
allx.to_csv('query_overlap.txt', encoding='utf-8')

In [166]:
#allx[(allx.dradis_clicks>=10)&(allx.js_clicks>=10)].sort_values('dradis_clicks', ascending=0).head(25)

In [172]:
fx = []
for i in list(allx.index):
    try:
        fx.append(str(i))
    except:
        print 'error'

error


In [193]:
fqry = ("FROM semgoogle 90d today WHERE account_country='US' "
        "report_origin='SEARCH_QUERY_PERFORMANCE_REPORT' query in {0} "
        "account_name=~'us_jobseeker.*' clicks>0 GROUP BY query, account_name, "
        "campaign_name, adgroup_name  SELECT clicks").format(tuple(fx))

In [196]:
sdf = iql.dataframe(fqry)

In [203]:
sdf.sort_values('clicks', ascending=0).to_csv('query_overlap_with_account_campaign_adgroup_info.txt', index=False, encoding='utf-8')

In [205]:
allx.js_clicks.sum()

72620

In [206]:
allx.dradis_clicks.sum()

208341

In [209]:
72620./1500000

0.048413333333333336